In [ ]:
#设计一个环境，环境是一个类，所以说我们要继承gymnasium.env这个类，他包含了所有的环境需要结构
#一个环境最基础的是需要定义一个观测空间和一个动作空间
from typing import Optional
import numpy as np
import gymnasium as gym
from torch._higher_order_ops.triton_kernel_wrap import triton_kernel_wrapper_functional_fake_tensor_mode
from torch.backends.cuda import flash_sdp_enabled

###################################################################
#定义了网格大小
#定义了初始位置和目标位置
#定义了观测空间
#定义了动作空间和动作的含义
##################################################################
class grid_env(gym.Env): #继承这个类有鸡毛用錒
    #size_cow size_rol 代表网格世界的大小，我们需要的是一个自定义长方形的
    def __init__(self,size_cow,size_rol):
        super().__init__()
        #定义环境的大小
        self.size_cow = size_cow
        self.size_rol = size_rol
        #定义了环境的大小后我们要定义我们智能体初始在的位置
        self._action_location = np.array([-1,-1],dtype=np.int32)
        self._target_location = np.array([-1,-1],dtype=np.int32)

        #我们已经把智能体的初始位置在环境中定好了，也就是智能体已经放到环境中去了
        #还有就是我们作为程序员通过代码知道的目前智能体的在空间的状态
        self.observation_space = gym.spaces.Dict({
            "agent":gym.spaces.Box(low=np.array([0,0]),
            high=np.array([self.size_cow-1,self.size_rol-1]),
            dtype=int),

            "target":gym.spaces.Box(low=np.array([0,0]),
            high=np.array([self.size_cow-1,self.size_rol-1]),
            dtype=int),
        })#意味着gym.state还有其他的定义方式，我们可以去space这个类里面去找
        
        #所以我们要定义我们智能体可以采取的动作
        self.action_space = gym.spaces.Discrete(4) #DIScrete这个函数啥意思呀？
        #定义我们具体的四个动作可以干什么
        self._action_to_direction = {
            #其实就是我如果采取了这个行为以后，得到的东西，这个得到的东西是为了智能体的移动
            0:np.array([1,0]), #向右
            1:np.array([0,1]), #向上
            2:np.array([-1,0]), #向左
            3:np.array([0,-1]), #向下
            #难道说这个数组列表是从下往上数行的吗？感觉这个向上和向下完全不能理解
        }
    #########################################这段代码的作用不好理解####################################################
    #写一个_get_obs这个辅助的函数来帮助我们将环境内部的状态改变成我们程序员观测的格式
    def _get_obs(self):
        return {
            "agent":self._agent_location,
            "target":self._target_location
        }
    #在调用env.step的时候会返回info，所以这个info我们可以自己去设定，需要返回什么info我们自己去设置就可以了
    def _get_info(self):
        return {
            "distance" : np.linalg.norm(self._agent_location - self._target_location,ord=1)
        }
    #################################################################################################################

    #reset() 方法启动一个新的回合,在我们的网格世界环境中，reset() 随机地将智能体和目标放置在网格上，确保它们不会在同一个位置开始。
    def reset(self,seed : Optional[int] = None,options : Optional[dict] = None):
        """Start a new episode.

        Args:
            seed: Random seed for reproducible episodes
            options: Additional configuration (unused in this example)

        Returns:
            tuple: (observ ation, info) for the initial state
        """
        # IMPORTANT: Must call this first to seed the random number generator
        super().reset(seed = seed)
        #随意的放置这个agent到任何一个网格的位置上，实际上这个位置是我们可以自定义的，可以放在OPtional里面来传入
        self._agent_location = self.np_random.integers(low=0,high=[self.size_cow,self.size_rol],size=2,dtype = int)
        self._target_location = self._agent_location
        while np.array_equal(self._agent_location,self._target_location):

            self._target_location = self.np_random.integers(low=0,high=np.array([self.size_cow,self.size_rol]),size=2,dtype = int)

        # #固定一个出发地和一个目标地
        # self._agent_location = np.array([2,0])
        # self._target_location = np.array([2,9])

        observation = self._get_obs()
        info = self._get_info()

        return observation,info
        #step() 方法包含核心环境逻辑。它接收一个动作，更新环境状态，并返回结果。物理、游戏规则和奖励逻辑都在这里。

        #对于网格世界，我们需要：
        # 1. 将离散动作转换为移动方向 
        # 2. 更新智能体的位置（带边界检查）
        # 3. 根据是否达到目标计算奖励 
        # 4. 判断回合是否应该结束 
        # 5. 返回所有所需信息

    def step(self,action):
        direction = self._action_to_direction[action]

        self._agent_location=np.clip(self._agent_location + direction,np.array([0,0]),np.array([self.size_cow-1,self.size_rol-1]))

        terminated = np.array_equal(self._agent_location,self._target_location)

        truncated = False
        distance = np.linalg.norm(self._agent_location - self._target_location)
        reward = 1 if terminated else -1*distance

        observation = self._get_obs()
        info = self._get_info()

        return observation, reward, terminated, truncated, info

from gymnasium.utils.env_checker import check_env

# This will catch many common issues
try:
    env = grid_env(5,10)
    check_env(env)
    print("Environment passes all checks!")
except Exception as e:
    print(f"Environment has issues: {e}")

Environment passes all checks!
